# Informe Comparativo de Cosecha de Datos: Phase 1 Improvements

## Objetivo
Comparar la calidad, volumen y cobertura de los datos recolectados **antes** de las mejoras de la Fase 1 (Old Data) vs. **después** de las mejoras (New Data).

**Alcance Temporal:** 2019 - 2023 (Periodo solapado)
**Datasets:**
- **Old:** `data/yape_{YEAR}.csv`
- **New:** `data/raw/yape_{YEAR}.csv`

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os
from pathlib import Path

# Configuración visual
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

## 1. Carga de Datos

In [ ]:
def load_dataset(base_path, label, years):
    dfs = []
    for year in years:
        pattern = os.path.join(base_path, f"yape_{year}*.csv")
        files = glob.glob(pattern)
        for f in files:
            if "gap" in f: continue # Skip gap files if any, to be fair
            try:
                # Intentar leer con detección automática de separador si falla
                try:
                    df = pd.read_csv(f)
                except:
                    df = pd.read_csv(f, sep=';')
                
                # Standardize columns
                # Legacy fields sometimes use 'published_at' or 'newspaper'
                if 'published_at' in df.columns:
                    df.rename(columns={'published_at': 'publish_date'}, inplace=True)
                if 'newspaper' in df.columns:
                    df.rename(columns={'newspaper': 'domain'}, inplace=True)
                if 'plain_text' in df.columns:
                    df.rename(columns={'plain_text': 'text'}, inplace=True)

                df['dataset'] = label
                df['year_file'] = year
                dfs.append(df)
            except Exception as e:
                print(f"Error reading {f}: {e}")
    
    if not dfs:
        return pd.DataFrame()
    
    full_df = pd.concat(dfs, ignore_index=True)
    return full_df

REQ_YEARS = [2019, 2020, 2021, 2022, 2023]
PROJECT_ROOT = Path("..") # Assuming notebook is in 'notebooks/'

old_df = load_dataset(PROJECT_ROOT / "data", "Old (Legacy)", REQ_YEARS)
new_df = load_dataset(PROJECT_ROOT / "data/raw", "New (Phase 1)", REQ_YEARS)

print(f"Old Dataset Total Rows: {len(old_df)}")
print(f"New Dataset Total Rows: {len(new_df)}")

combined_df = pd.concat([old_df, new_df], ignore_index=True)

## 2. Estandarización y Limpieza

In [ ]:
# Convertir fechas
# Ensure publish_date is datetime
combined_df['publish_date'] = pd.to_datetime(combined_df['publish_date'], errors='coerce', utc=True)
combined_df['month_year'] = combined_df['publish_date'].dt.to_period('M')

# Fill NA source_api for old data if missing
if 'source_api' not in combined_df.columns:
    combined_df['source_api'] = 'Unknown'
combined_df['source_api'] = combined_df['source_api'].fillna('Legacy')

combined_df.head()

## 3. Análisis Volumétrico

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(data=combined_df, x='year_file', hue='dataset', palette='viridis')
plt.title("Comparación de Volumen de Artículos por Año")
plt.ylabel("Número de Artículos")
plt.xlabel("Año")
plt.show()

In [ ]:
# Distribución mensual
monthly_counts = combined_df.groupby(['month_year', 'dataset']).size().reset_index(name='count')
monthly_counts['month_year'] = monthly_counts['month_year'].astype(str)

plt.figure(figsize=(15, 6))
sns.lineplot(data=monthly_counts, x='month_year', y='count', hue='dataset', marker='o')
plt.xticks(rotation=90)
plt.title("Tendencia Mensual de Recolección (2019-2023)")
plt.tight_layout()
plt.show()

## 4. Análisis de Contenido (Calidad)

In [ ]:
# Calcular longitud del texto (si existe la columna de texto)
# Posibles nombres: 'text', 'content', 'body', 'plain_text'
text_col = 'text' if 'text' in combined_df.columns else 'content'
if text_col not in combined_df.columns:
    # Fallback search
    for col in combined_df.columns:
        if 'text' in col or 'content' in col:
            text_col = col
            break

print(f"Usando columna de texto: {text_col}")

combined_df['text_length'] = combined_df[text_col].astype(str).apply(len)

# Plot distribución de longitud
plt.figure(figsize=(12, 6))
sns.boxplot(data=combined_df, x='dataset', y='text_length', showfliers=False)
plt.title("Distribución de Longitud de Texto (Sin Outliers Extremos)")
plt.show()

# Estadísticas descriptivas de longitud
combined_df.groupby('dataset')['text_length'].describe()

## 5. Análisis de Fuentes y Dominios

In [ ]:
top_domains_old = old_df['domain'].value_counts().head(10)
top_domains_new = new_df['domain'].value_counts().head(10)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sns.barplot(x=top_domains_old.values, y=top_domains_old.index, ax=axes[0], palette="Blues_r")
axes[0].set_title("Top 10 Dominios - OLD Dataset")

sns.barplot(x=top_domains_new.values, y=top_domains_new.index, ax=axes[1], palette="Greens_r")
axes[1].set_title("Top 10 Dominios - NEW Dataset")

plt.tight_layout()
plt.show()

## 6. Overlap y Duplicidad
¿Cuántos artículos coinciden por URL entre ambos datasets?

In [ ]:
common_urls = set(old_df['url']).intersection(set(new_df['url']))
overlap_pct_old = len(common_urls) / len(old_df) * 100
overlap_pct_new = len(common_urls) / len(new_df) * 100

print(f"Artículos comunes (match por URL): {len(common_urls)}")
print(f"Porcentaje del OLD cubierto en NEW: {overlap_pct_old:.2f}%")
print(f"Porcentaje del NEW presente en OLD: {overlap_pct_new:.2f}%")

# Venn Diagram concept (simple text)
unique_old = len(old_df) - len(common_urls)
unique_new = len(new_df) - len(common_urls)

print(f"\nExclusivos en OLD: {unique_old}")
print(f"Exclusivos en NEW: {unique_new}")

## 7. Conclusión y Recomendación de Merge

Basado en los datos anteriores, evaluamos si fusionar es beneficioso.

**Criterios:**
1. Si `Exclusivos en OLD` es alto (> 10-20%), vale la pena fusionar para no perder histórico.
2. Si la calidad del texto en OLD es muy inferior (ver boxplot longitud), quizás sea mejor descartarlo o reprocesar sus URLs.

In [ ]:
# Simulación de Merge Estratégico
# Prioridad: NEW (Phase 1) porque tiene mejor limpieza de texto.
# Relleno: Artículos de OLD que NO estén en NEW.

merged_df = pd.concat([
    new_df, 
    old_df[~old_df['url'].isin(new_df['url'])]
], ignore_index=True)

print(f"Dataset Fusionado Potencial: {len(merged_df)} registros.")
print(f"Ganancia Neta sobre NEW: {len(merged_df) - len(new_df)} registros.")